# Nonadiabatic Dynamics with Shin-Metiu Polaritonic Hamiltonians


## Table of Content: <a name="TOC"></a>

1. [Generic setups](#1)   

2. [Define polariton model parameters](#2)

3. [Choose the initial conditions: Nuclear and Electronic](#3)

5. [Running the Simulation](#4)

6. [Plotting the results](#5)



## A. Learning objectives

- Define model and dynamics parameters
- Initialize nuclear and electronic degrees of freedom
- Run the Ehrenfest dynamics (with other TSH methods as options) in polaritonic basis
- Plot the resulting dynamics


## B. Use cases

- Shin-Metiu Polariton dyanmics


## C. Functions


- libra_py
  - models
    - Shin_Metiu
      - polariton
        - [compute_model](#compute_model-1)
  - dynamics
    - `tsh`
      - `compute`
        - [`generic_recipe`](#generic_recipe-1)
      - `plot`
        - [`plot_dynamics`](#plot_dynamics-1)     

## D. Classes and class members


## 1. Generic setups
<a name="1"></a>[Back to TOC](#TOC)

Here, we import all necessary libraries, set up some definitions (e.g. colors), and define the function that would be calling model Hamiltonians also defined within Libra package.

In [ ]:
#Importing Required Libraries
import math
import numpy as np
import warnings
from functools import reduce
import matplotlib.pyplot as plt   # plots

# Core simulation modules
from libra_py.models.Shin_Metiu import polariton
from libra_py.models.Shin_Metiu.polariton import compute_model
from liblibra_core import *
import util.libutil as comn
from libra_py import units
import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot

# Specific dynamics recipes
from recipes import ehrenfest_adi_ld, ehrenfest_dia, ehrenfest_adi_nac, fssh

# Ignore warnings for cleaner output
warnings.filterwarnings('ignore')

colors = {}
colors.update({"11": "#8b1a0e"})  # red
colors.update({"12": "#FF4500"})  # orangered
colors.update({"13": "#B22222"})  # firebrick
colors.update({"14": "#DC143C"})  # crimson
colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive
colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue
colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

## 2. Define polariton model parameters
<a name="2"></a>[Back to TOC](#TOC)

We define all important parameters for the Shin-Metiu polaritonic models.


Here we define the default parameters based on J. Chem. Phys. 157, 104118 (2022) paper.

In [ ]:
# Number of electronic states in the simulation
nstates = 4
# Initial electronic state index
n_init = 1

# Define polariton model parameters
model_params = {
    "model": 1,         # model 0: SM1 model , model 1: SM2 model
    "model0": 1,
    "ndof": 1,          # Number of nuclear degrees of freedom
    "g_c": 0.005,       # Light-matter coupling
    "omega_c": 0.1,     # Cavity frequency (a.u.)
    "epsilon": 1.0,     # Dielectric constant
    "nstates": nstates
}


In [ ]:
# Main dynamics parameters dictionary
dyn_general = {
    "nsteps": 500,                   # Total time steps
    "ntraj": 2000,                   # Number of trajectories
    "nstates": nstates,              # Electronic states
    "dt": 0.1 * units.fs2au,         # Time step (in a.u.)
    "num_electronic_substeps": 1,    # Substeps per nuclear step
    "isNBRA": 0, "is_nbra": 0,       # Disable NBRA
    "progress_frequency": 0.1,       # Progress printing frequency
    "which_adi_states": range(nstates),
    "which_dia_states": range(nstates),
    "mem_output_level": 3,           # Memory output detail level
    "properties_to_save": [
        "timestep", "time", "q", "p", "f", "Cadi", "Cdia",
        "Epot_ave", "Ekin_ave", "Etot_ave",
        "se_pop_adi", "se_pop_dia",
        "sh_pop_adi", "sh_pop_dia",
        "mash_pop_adi", "mash_pop_dia"
    ],
    "prefix": "adiabatic_md",
    "prefix2": "adiabatic_md"
}

# Load the specific Ehrenfest recipe (adiabatic NACs)
# One can choose other receipes
# fssh.load(dyn_general)
#ehrenfest_adi_ld.load(dyn_general)
# ehrenfest_dia.load(dyn_general)
ehrenfest_adi_nac.load(dyn_general)

## 3. Choose the initial conditions: Nuclear and Electronic
<a name="3"></a>[Back to TOC](#TOC)

 we initialize nuclear DOFs based on J. Chem. Phys. 157, 104118 (2022) paper.

In [ ]:
# Initialization type for nuclear DOFs:
# 0: fixed coords and momenta
# 1: fixed coords, sampled momenta
# 2: sampled coords, fixed momenta
# 3: both coords and momenta sampled
# Here the paramters are set according to the J. Chem. Phys. 157, 104118 (2022) paper
icond_nucl = 3

nucl_params = {
    "ndof": 1,
    "q": [-4],                       # Initial position
    "p": [0.0],                      # Initial momentum
    "mass": [1836.0],                 # Proton mass in a.u.
    "force_constant": [0.000382**2 * 1836.0],
    "init_type": icond_nucl
}


 we initialize electronic DOFs based on J. Chem. Phys. 157, 104118 (2022) paper.

In [ ]:
#Initial Conditions：Electronic DOFs
# Representation:
# 0: diabatic wavefunctions
# 1: adiabatic wavefunctions
rep = 0

# Initial population vector (all zeros except chosen initial state)
istates = [0.0] * nstates
istates[n_init] = 1.0

elec_params = {
    "verbosity": 2,
    "init_dm_type": 0,
    "ndia": nstates,
    "nadi": nstates,
    "rep": rep,
    "init_type": 0,
    "istates": istates,
    "istate": n_init
}


## 4. Running the Simulation
<a name="4"></a>[Back to TOC](#TOC)

Running dynamics

In [ ]:
# Running the Simulation
# Clone the general dynamics parameters for the specific run
dyn_params = dict(dyn_general)

# Create a random number generator for stochastic parts
rnd = Random()

# Run the Ehrenfest simulation
res = tsh_dynamics.generic_recipe(
    dyn_params,
    compute_model,
    model_params,
    elec_params,
    nucl_params,
    rnd
)


## 5. Plotting the results
<a name="5"></a>[Back to TOC](#TOC)

Plotting the quantities

In [ ]:
#Plotting the Results
plot_params = {
    "prefix": "adiabatic_md",
    "filename": "mem_data.hdf",
    "output_level": 3,
    "which_trajectories": [0],
    "which_dofs": [0],
    "which_adi_states": list(range(nstates)),
    "which_dia_states": list(range(nstates)),
    "frameon": True,
    "linewidth": 3,
    "dpi": 300,
    "axes_label_fontsize": (8,8),
    "legend_fontsize": 8,
    "axes_fontsize": (8,8),
    "title_fontsize": 8,
    "what_to_plot": [
        "coordinates", "momenta",  "forces", "energies", "phase_space",
        "se_pop_adi", "se_pop_dia", "sh_pop_adi", "sh_pop_dia",
        "mash_pop_adi", "mash_pop_dia"
    ],
    "which_energies": ["potential", "kinetic", "total"],
    "save_figures": 1,
    "do_show": 1
}

tsh_dynamics_plot.plot_dynamics(plot_params)